In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
import os 
os.chdir(r"C:\Users\00059700\Desktop\Займы\new_files")

# ipoteka = pd.read_csv("ipoteka.csv", sep=';', dtype=str)

# avtotransport = pd.read_csv("avtotransport.csv", sep=';',dtype=str)

# potreb = pd.read_csv("potreb.csv", sep=';')

avtotransport = pd.read_csv("obrazov_pz_pension.csv", sep=';', dtype=str)



## ипотека = 4,2 

In [2]:
ipoteka

,DATE_VALUE,SUBJECT_IIN_BIN,RS_CR_CONTRACT_ID,CONTRACT_NUMBER,CONTRACT_PRODUCT_CODE,CONTRACT_PRODUCT_NAME,CONTRACT_OPEN_DATE,CONTRACT_END_DATE,CONTRACT_LIQUIDATE_DATE,LOAN_TERM_DAY_PLAN,...,CR_EXP_IN_NAT,CR_RES_DZ_MSFO_NAT,CR_RES_MSFO_NAT,DATE_VALUE.1,SHEDULED_SUM_NAT,PLAN_DATE,FACT_DATE,STATUS,PAY_TYPE,PLAN_DATE_M
0,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,NaN,NaN,NaN,11.09.2024,"4999,74",04.11.2024,NaN,NaN,CR_INTER,01.11.2024
1,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,NaN,NaN,NaN,11.09.2024,"736,72",04.06.2026,NaN,NaN,CR_INTER,01.06.2026
2,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,NaN,NaN,NaN,11.09.2024,"2613,9",06.10.2025,NaN,NaN,CR_INTER,01.10.2025
3,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,NaN,NaN,NaN,11.09.2024,"2386,01",04.11.2025,NaN,NaN,CR_INTER,01.11.2025
4,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,NaN,NaN,NaN,11.09.2024,"247,63",04.08.2026,NaN,NaN,CR_INTER,01.08.2026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11812069,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,NaN,NaN,"90786,45",11.09.2024,"95123,42",12.11.2035,NaN,NaN,CR_PD,01.11.2035
11812070,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,NaN,NaN,"90786,45",11.09.2024,"66595,21",12.03.2030,NaN,NaN,CR_INTER,01.03.2030
11812071,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,NaN,NaN,"90786,45",11.09.2024,"83622,8",12.12.2034,NaN,NaN,CR_PD,01.12.2034
11812072,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,NaN,NaN,"90786,45",11.09.2024,"45611,05",12.01.2033,NaN,NaN,CR_INTER,01.01.2033


In [5]:
ipoteka['PLAN_DATE'] = pd.to_datetime(ipoteka['PLAN_DATE'], format='%d.%m.%Y')

# Convert the date format to 'dd.mm.yyyy'
ipoteka['PLAN_DATE'] = ipoteka['PLAN_DATE'].dt.strftime('%d.%m.%Y')
ipoteka['PLAN_DATE'] = pd.to_datetime(ipoteka['PLAN_DATE'], format='%d.%m.%Y')
ipoteka['REPORT_DATE']='01.09.2024'
ipoteka['REPORT_DATE'] = pd.to_datetime(ipoteka['REPORT_DATE'], format='%d.%m.%Y')

# Calculate the time difference in years
ipoteka['time_to_mat'] = (ipoteka['PLAN_DATE'] - ipoteka['REPORT_DATE']).dt.days / 365
ipoteka['EFF_INTEREST_RATE']=ipoteka['EFF_INTEREST_RATE'].str.replace(',','.')
ipoteka['EFF_INTEREST_RATE']=ipoteka['EFF_INTEREST_RATE'].astype(float)
ipoteka['disc_fact']=1/(1+ipoteka.EFF_INTEREST_RATE/100)**ipoteka.time_to_mat
ipoteka['SHEDULED_SUM_NAT']=ipoteka['SHEDULED_SUM_NAT'].str.replace(',','.')
ipoteka['SHEDULED_SUM_NAT']=ipoteka['SHEDULED_SUM_NAT'].astype(float)
ipoteka['MP*disc_fact']=ipoteka.SHEDULED_SUM_NAT*ipoteka.disc_fact
ipoteka['MP*disc_fact*time']=ipoteka['MP*disc_fact']*ipoteka.time_to_mat
ipoteka['CR_BAL_NAT']=ipoteka['CR_BAL_NAT'].str.replace(',','.')
ipoteka['CR_BAL_NAT'] = ipoteka['CR_BAL_NAT'].replace('', '0')

# Convert the column to float
ipoteka['CR_BAL_NAT'] = ipoteka['CR_BAL_NAT'].astype(float)
ipoteka_number=ipoteka.groupby(['RS_CR_CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
ipoteka_number['duration']=ipoteka_number['MP*disc_fact*time']/ipoteka_number['MP*disc_fact']
ipoteka_number['mod_dur']=ipoteka_number['duration']/(1+ipoteka.EFF_INTEREST_RATE/100)
ipoteka_number.CR_BAL_NAT=ipoteka_number.CR_BAL_NAT*(-1)
ipoteka_number['weighted']=ipoteka_number.mod_dur*ipoteka_number.CR_BAL_NAT
ipoteka_number['weighted']=ipoteka_number['weighted'].fillna(0)
sum(ipoteka_number['weighted'])/sum(ipoteka_number.CR_BAL_NAT)

4.218128491060724

In [14]:
pd.set_option('display.max_columns', None)


In [8]:
ipoteka.head()

,DATE_VALUE,SUBJECT_IIN_BIN,RS_CR_CONTRACT_ID,CONTRACT_NUMBER,CONTRACT_PRODUCT_CODE,CONTRACT_PRODUCT_NAME,CONTRACT_OPEN_DATE,CONTRACT_END_DATE,CONTRACT_LIQUIDATE_DATE,LOAN_TERM_DAY_PLAN,LOAN_TERM_MONTH,CONTRACT_DURATION,CURRENCY_CODE,CONTRACT_SUM_NAT,INTEREST_RATE,EFF_INTEREST_RATE,CONTRACT_STATUS_NAME,EXP_PD_OVRD_CNT,CR_BAL_NAT,CR_EXP_PD_NAT,CR_INT_CH_NAT,CR_EXP_IN_NAT,CR_RES_DZ_MSFO_NAT,CR_RES_MSFO_NAT,DATE_VALUE.1,SHEDULED_SUM_NAT,PLAN_DATE,FACT_DATE,STATUS,PAY_TYPE,PLAN_DATE_M,REPORT_DATE,time_to_mat,disc_fact,MP*disc_fact,MP*disc_fact*time
0,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,240,2487,KZT,3104654,10,0.0,Продан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.09.2024,4999.74,2024-11-04,NaN,NaN,CR_INTER,01.11.2024,2024-09-01,0.175342,1.0,4999.74,876.666740
1,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,240,2487,KZT,3104654,10,0.0,Продан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.09.2024,736.72,2026-06-04,NaN,NaN,CR_INTER,01.06.2026,2024-09-01,1.756164,1.0,736.72,1293.801425
2,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,240,2487,KZT,3104654,10,0.0,Продан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.09.2024,2613.90,2025-10-06,NaN,NaN,CR_INTER,01.10.2025,2024-09-01,1.095890,1.0,2613.90,2864.547945
3,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,240,2487,KZT,3104654,10,0.0,Продан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.09.2024,2386.01,2025-11-04,NaN,NaN,CR_INTER,01.11.2025,2024-09-01,1.175342,1.0,2386.01,2804.378877
4,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,240,2487,KZT,3104654,10,0.0,Продан,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.09.2024,247.63,2026-08-04,NaN,NaN,CR_INTER,01.08.2026,2024-09-01,1.923288,1.0,247.63,476.263726


In [10]:
ipoteka.EFF_INTEREST_RATE.value_counts()

EFF_INTEREST_RATE
7.20     2551526
19.20     838867
3.10      604478
11.30     481108
3.00      429135
          ...   
4.70          17
32.90         16
52.50         15
11.68          8
7.60           4
Name: count, Length: 314, dtype: int64

In [11]:
ipoteka_number

,RS_CR_CONTRACT_ID,EFF_INTEREST_RATE,MP*disc_fact,MP*disc_fact*time,CR_BAL_NAT,duration,mod_dur,weighted
0,36468090,3.0,1.403247e+06,7.818709e+06,1390061.16,5.571868,5.571868,7.745237e+06
1,36468142,14.4,3.189527e+06,1.324025e+07,3178345.16,4.151163,4.151163,1.319383e+07
2,36468153,14.0,2.000743e+06,5.156496e+06,1998276.21,2.577291,2.577291,5.150139e+06
3,36468196,3.0,2.943923e+06,1.598327e+07,2934234.63,5.429242,5.429242,1.593067e+07
4,36468318,3.0,6.539816e+06,3.779188e+07,6513684.92,5.778737,5.778737,3.764087e+07
...,...,...,...,...,...,...,...,...
44060,46569316,12.7,1.439050e+07,9.176527e+07,14400000.00,6.376796,5.376725,7.742485e+07
44061,46569428,19.2,1.082415e+07,5.533507e+07,10924000.00,5.112188,4.310445,4.708730e+07
44062,46569619,19.9,3.475782e+07,1.074848e+08,35035000.00,3.092392,2.607413,9.135072e+07
44063,46569680,19.3,8.063451e+06,4.107858e+07,8173072.00,5.094417,4.295461,3.510711e+07


In [6]:
ipoteka['disc_fact']=1/(1+ipoteka.EFF_INTEREST_RATE/100)**ipoteka.time_to_mat

ipoteka['SHEDULED_SUM_NAT']=ipoteka['SHEDULED_SUM_NAT'].str.replace(',','.')
ipoteka['SHEDULED_SUM_NAT']=ipoteka['SHEDULED_SUM_NAT'].astype(float)

ipoteka['MP*disc_fact']=ipoteka.SHEDULED_SUM_NAT*ipoteka.disc_fact
ipoteka['MP*disc_fact*time']=ipoteka['MP*disc_fact']*ipoteka.time_to_mat


ipoteka['CR_BAL_NAT']=ipoteka['CR_BAL_NAT'].str.replace(',','.')
ipoteka['CR_BAL_NAT'] = ipoteka['CR_BAL_NAT'].replace('', '0')
ipoteka['CR_BAL_NAT'] = ipoteka['CR_BAL_NAT'].astype(float)


ipoteka_number=ipoteka.groupby(['RS_CR_CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
ipoteka_number['duration']=ipoteka_number['MP*disc_fact*time']/ipoteka_number['MP*disc_fact']
ipoteka_number['mod_dur']=ipoteka_number['duration']/(1+ipoteka.EFF_INTEREST_RATE/100)
ipoteka_number.CR_BAL_NAT=ipoteka_number.CR_BAL_NAT*(-1)
ipoteka_number['weighted']=ipoteka_number.mod_dur*ipoteka_number.CR_BAL_NAT
ipoteka_number['weighted']=ipoteka_number['weighted'].fillna(0)



,DATE_VALUE,SUBJECT_IIN_BIN,RS_CR_CONTRACT_ID,CONTRACT_NUMBER,CONTRACT_PRODUCT_CODE,CONTRACT_PRODUCT_NAME,CONTRACT_OPEN_DATE,CONTRACT_END_DATE,CONTRACT_LIQUIDATE_DATE,LOAN_TERM_DAY_PLAN,...,PLAN_DATE,FACT_DATE,STATUS,PAY_TYPE,PLAN_DATE_M,REPORT_DATE,time_to_mat,disc_fact,MP*disc_fact,MP*disc_fact*time
0,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,2024-11-04,NaN,NaN,CR_INTER,01.11.2024,2024-09-01,0.175342,1.000000,4999.740000,876.666740
1,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,2026-06-04,NaN,NaN,CR_INTER,01.06.2026,2024-09-01,1.756164,1.000000,736.720000,1293.801425
2,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,2025-10-06,NaN,NaN,CR_INTER,01.10.2025,2024-09-01,1.095890,1.000000,2613.900000,2864.547945
3,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,2025-11-04,NaN,NaN,CR_INTER,01.11.2025,2024-09-01,1.175342,1.000000,2386.010000,2804.378877
4,31.08.2024,781007400711,38728393,1199781000058194,16119,Ипотечный займ по спецпрограмме АО «КИК»,02.08.2006,04.08.2026,NaN,7307,...,2026-08-04,NaN,NaN,CR_INTER,01.08.2026,2024-09-01,1.923288,1.000000,247.630000,476.263726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11812069,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,2035-11-12,NaN,NaN,CR_PD,01.11.2035,2024-09-01,11.202740,0.202928,19303.222612,216248.978792
11812070,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,2030-03-12,NaN,NaN,CR_INTER,01.03.2030,2024-09-01,5.528767,0.455156,30311.201479,167583.574205
11812071,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,2034-12-12,NaN,NaN,CR_PD,01.12.2034,2024-09-01,10.284932,0.231254,19338.131744,198891.360455
11812072,31.08.2024,980313350794,41341597,KZ43914111417KZ00M17-11,16431,Ипотека Сбербанк,07.10.2021,07.10.2036,NaN,5479,...,2033-01-12,NaN,NaN,CR_INTER,01.01.2033,2024-09-01,8.369863,0.303737,13853.746542,115953.960785


In [12]:
sum(ipoteka_number['weighted'])/sum(ipoteka_number.CR_BAL_NAT)


4.218128491060724

## АВТОТРАНСПОРТ = 1,9%

In [2]:
avtotransport

,DATE_VALUE,SUBJECT_IIN_BIN,RS_CR_CONTRACT_ID,CONTRACT_NUMBER,CONTRACT_PRODUCT_CODE,CONTRACT_PRODUCT_NAME,CONTRACT_OPEN_DATE,CONTRACT_END_DATE,CONTRACT_LIQUIDATE_DATE,LOAN_TERM_DAY_PLAN,...,CR_EXP_IN_NAT,CR_RES_DZ_MSFO_NAT,CR_RES_MSFO_NAT,DATE_VALUE.1,SHEDULED_SUM_NAT,PLAN_DATE,FACT_DATE,STATUS,PAY_TYPE,PLAN_DATE_M
0,31.08.2024,781007400711,46226589,1177770000250767,16158,На приобретение автотранспорта (автосалоны пар...,12.07.2024,12.07.2031,NaN,2556,...,NaN,NaN,"28086,25",11.09.2024,"119857,14",21.05.2025,NaN,NaN,CR_INTER,01.05.2025
1,31.08.2024,781007400711,46226589,1177770000250767,16158,На приобретение автотранспорта (автосалоны пар...,12.07.2024,12.07.2031,NaN,2556,...,NaN,NaN,"28086,25",11.09.2024,"97059,53",24.03.2026,NaN,NaN,CR_PD,01.03.2026
2,31.08.2024,781007400711,46226589,1177770000250767,16158,На приобретение автотранспорта (автосалоны пар...,12.07.2024,12.07.2031,NaN,2556,...,NaN,NaN,"28086,25",11.09.2024,"177412,86",22.07.2030,NaN,NaN,CR_PD,01.07.2030
3,31.08.2024,781007400711,46226589,1177770000250767,16158,На приобретение автотранспорта (автосалоны пар...,12.07.2024,12.07.2031,NaN,2556,...,NaN,NaN,"28086,25",11.09.2024,"93740,2",22.12.2025,NaN,NaN,CR_PD,01.12.2025
4,31.08.2024,781007400711,46226589,1177770000250767,16158,На приобретение автотранспорта (автосалоны пар...,12.07.2024,12.07.2031,NaN,2556,...,NaN,NaN,"28086,25",11.09.2024,"110268,14",22.02.2027,NaN,NaN,CR_PD,01.02.2027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5015162,31.08.2024,941221351133,44626672,1199830000007814,16161,Льготный автокредит НФ РК,27.11.2023,27.11.2030,NaN,2557,...,NaN,NaN,"25728,93",11.09.2024,"111576,02",05.11.2025,NaN,NaN,CR_PD,01.11.2025
5015163,31.08.2024,941221351133,44626672,1199830000007814,16161,Льготный автокредит НФ РК,27.11.2023,27.11.2030,NaN,2557,...,NaN,NaN,"25728,93",11.09.2024,"120051,11",06.09.2027,NaN,NaN,CR_PD,01.09.2027
5015164,31.08.2024,941221351133,44626672,1199830000007814,16161,Льготный автокредит НФ РК,27.11.2023,27.11.2030,NaN,2557,...,NaN,NaN,"25728,93",11.09.2024,"30191,03",05.09.2024,05.09.2024,Выплачен,CR_INTER,01.09.2024
5015165,31.08.2024,941221351133,44626672,1199830000007814,16161,Льготный автокредит НФ РК,27.11.2023,27.11.2030,NaN,2557,...,NaN,NaN,"25728,93",11.09.2024,"23616,9",05.03.2026,NaN,NaN,CR_INTER,01.03.2026


In [3]:
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['PLAN_DATE'] = avtotransport['PLAN_DATE'].dt.strftime('%d.%m.%Y')
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['REPORT_DATE']='01.09.2024'
avtotransport['REPORT_DATE'] = pd.to_datetime(avtotransport['REPORT_DATE'], format='%d.%m.%Y')

# Calculate the time difference in years
avtotransport['time_to_mat'] = (avtotransport['PLAN_DATE'] - avtotransport['REPORT_DATE']).dt.days / 365
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].str.replace(',','.')
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].astype(float)
avtotransport['disc_fact']=1/(1+avtotransport.EFF_INTEREST_RATE/100)**avtotransport.time_to_mat
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].str.replace(',','.')
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].astype(float)
avtotransport['MP*disc_fact']=avtotransport.SHEDULED_SUM_NAT*avtotransport.disc_fact
avtotransport['MP*disc_fact*time']=avtotransport['MP*disc_fact']*avtotransport.time_to_mat
avtotransport['CR_BAL_NAT']=avtotransport['CR_BAL_NAT'].str.replace(',','.')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].replace('', '0')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].astype(float)

avtotransport_number=avtotransport.groupby(['RS_CR_CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
avtotransport_number['duration']=avtotransport_number['MP*disc_fact*time']/avtotransport_number['MP*disc_fact']
avtotransport_number['mod_dur']=avtotransport_number['duration']/(1+avtotransport.EFF_INTEREST_RATE/100)
avtotransport_number.CR_BAL_NAT=avtotransport_number.CR_BAL_NAT*(-1)
avtotransport_number['weighted']=avtotransport_number.mod_dur*avtotransport_number.CR_BAL_NAT
avtotransport_number['weighted']=avtotransport_number['weighted'].fillna(0)
sum(avtotransport_number['weighted'])/sum(avtotransport_number.CR_BAL_NAT)

1.9199496866509012

In [4]:
avtotransport_number

,RS_CR_CONTRACT_ID,EFF_INTEREST_RATE,MP*disc_fact,MP*disc_fact*time,CR_BAL_NAT,duration,mod_dur,weighted
0,36476156,7.2,1.978917e+06,1.951665e+06,2035257.25,0.986228,0.858336,1.746935e+06
1,36478756,5.9,1.294641e+06,1.250247e+06,1314286.00,0.965710,0.840479,1.104629e+06
2,36478886,6.1,1.712846e+06,1.854118e+06,1745999.81,1.082477,0.942104,1.644913e+06
3,36483346,7.1,4.683295e+05,8.573831e+04,469455.17,0.183073,0.159332,7.479930e+04
4,36486137,7.0,3.542128e+05,1.234294e+05,357422.82,0.348461,0.303274,1.083969e+05
...,...,...,...,...,...,...,...,...
45420,46581087,13.7,5.546428e+06,1.012931e+07,5545000.00,1.826277,1.771365,9.822218e+06
45421,46581108,17.8,8.394103e+06,2.410344e+07,8390000.00,2.871473,2.785134,2.336728e+07
45422,46581127,6.2,6.688390e+06,9.657446e+06,6690000.00,1.443912,1.400496,9.369322e+06
45423,46581165,6.4,7.195926e+06,1.057872e+07,7196656.00,1.470099,1.425896,1.026168e+07


In [3]:
obrazov_pz_pension.CONTRACT_PRODUCT_CODE.value_counts()

CONTRACT_PRODUCT_CODE
16111    1580
16136     204
16137     116
Name: count, dtype: int64

## Образование/Под Залог/ Пенсия 

In [5]:
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['PLAN_DATE'] = avtotransport['PLAN_DATE'].dt.strftime('%d.%m.%Y')
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['REPORT_DATE']='01.09.2024'
avtotransport['REPORT_DATE'] = pd.to_datetime(avtotransport['REPORT_DATE'], format='%d.%m.%Y')

# Calculate the time difference in years
avtotransport['time_to_mat'] = (avtotransport['PLAN_DATE'] - avtotransport['REPORT_DATE']).dt.days / 365
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].str.replace(',','.')
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].astype(float)
avtotransport['disc_fact']=1/(1+avtotransport.EFF_INTEREST_RATE/100)**avtotransport.time_to_mat
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].str.replace(',','.')
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].astype(float)
avtotransport['MP*disc_fact']=avtotransport.SHEDULED_SUM_NAT*avtotransport.disc_fact
avtotransport['MP*disc_fact*time']=avtotransport['MP*disc_fact']*avtotransport.time_to_mat
avtotransport['CR_BAL_NAT']=avtotransport['CR_BAL_NAT'].str.replace(',','.')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].replace('', '0')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].astype(float)

avtotransport_number=avtotransport.groupby(['CONTRACT_PRODUCT_CODE','RS_CR_CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
avtotransport_number['duration']=avtotransport_number['MP*disc_fact*time']/avtotransport_number['MP*disc_fact']
avtotransport_number['mod_dur']=avtotransport_number['duration']/(1+avtotransport.EFF_INTEREST_RATE/100)
avtotransport_number.CR_BAL_NAT=avtotransport_number.CR_BAL_NAT*(-1)
avtotransport_number['weighted']=avtotransport_number.mod_dur*avtotransport_number.CR_BAL_NAT
avtotransport_number['weighted']=avtotransport_number['weighted'].fillna(0)
sum(avtotransport_number['weighted'])/sum(avtotransport_number.CR_BAL_NAT)

1.9623524390224638

In [8]:
podzalog=avtotransport_number[avtotransport_number['CONTRACT_PRODUCT_CODE']=='16111']
sum(podzalog['weighted'])/sum(podzalog.CR_BAL_NAT)

1.9657436677134614

In [9]:
obrazov=avtotransport_number[avtotransport_number['CONTRACT_PRODUCT_CODE']=='16136']
sum(obrazov['weighted'])/sum(obrazov.CR_BAL_NAT)

0.4889768106909127

In [11]:
pension=avtotransport_number[avtotransport_number['CONTRACT_PRODUCT_CODE']=='16137']
sum(pension['weighted'])/sum(pension.CR_BAL_NAT)

0.5896526553303174

In [ ]:
import pandas as pd
import os 

os.chdir(r"C:\Users\00059700\Desktop\Займы\new_files")

# Read the total number of rows in the file
total_rows = sum(1 for line in open('potreb.csv', encoding='utf-8'))

# Calculate the number of rows to skip (50% of total rows)
skip_rows = total_rows // 2

# Read only the first 50% of the file
avtotransport = pd.read_csv("potreb.csv", encoding='utf-8', sep=';', skiprows=range(1, skip_rows))

# Now 'potreb' contains the first 50% of the data from "potreb.csv"

C:\Users\00059700\AppData\Local\Temp\ipykernel_21416\3178453551.py:13: DtypeWarning: Columns (3,13) have mixed types. Specify dtype option on import or set low_memory=False.
  avtotransport = pd.read_csv("potreb.csv", encoding='utf-8', sep=';', skiprows=range(1, skip_rows))


## BS

In [5]:
import os 
import pandas as pd
os.chdir(r"C:\Users\00059700\Desktop\Займы\new_files")

avtotransport = pd.read_csv("BS_ZAIMI.csv", sep=';', dtype=str)


In [6]:
avtotransport.SUBJECT_BUSINESS_TYPE_NAME.value_counts()

SUBJECT_BUSINESS_TYPE_NAME
Малый бизнес              11767527
Средний бизнес             3499838
Корпоративный бизнес 1     1887941
Корпоративный бизнес 2     1656729
Дочерние организации         40930
Розничный бизнес               130
Name: count, dtype: int64

In [27]:
1887941+1656729

3544670

In [22]:
avtotransport[avtotransport['SUBJECT_BUSINESS_TYPE_NAME'].isna()]

,DATE_VALUE,SUBJECT_IIN_BIN,SUBJECT_BUSINESS_TYPE_NAME,CONTRACT_ID,CONTRACT_NUMBER,CONTRACT_PRODUCT_CODE,CONTRACT_PRODUCT_NAME,CONTRACT_OPEN_DATE,CONTRACT_END_DATE,CONTRACT_LIQUIDATE_DATE,REMAIN_TERM,REPAYMENT_DAY,CONTRACT_DURATION,CURRENCY_CODE,CONTRACT_SUM_NAT,INTEREST_RATE,EFF_INTEREST_RATE,CONTRACT_STATUS_NAME,EXP_PD_OVRD_CNT,CR_BAL_NAT,CR_EXP_PD_NAT,CR_INT_CH_NAT,CR_EXP_IN_NAT,DATE_VALUE.1,SHEDULED_SUM_NAT,PLAN_DATE,FACT_DATE,STATUS,PAY_TYPE,PLAN_DATE_M
373754,31.08.2024,120140004013,NaN,46538725,OBZ_D73433843865,10475,Кредиты в нацвалюте юридическим лицам с возмож...,26.08.2024,26.08.2026,NaN,24 месяца,15,355,KZT,4687000,"25,4","28,6",Актуален,NaN,4687000,NaN,"-13227,76",NaN,10.09.2024,"234190,17",15.06.2026,NaN,NaN,CR_PD,01.06.2026
373755,31.08.2024,120140004013,NaN,46538725,OBZ_D73433843865,10475,Кредиты в нацвалюте юридическим лицам с возмож...,26.08.2024,26.08.2026,NaN,24 месяца,15,355,KZT,4687000,"25,4","28,6",Актуален,NaN,4687000,NaN,"-13227,76",NaN,10.09.2024,"229335,9",15.05.2026,NaN,NaN,CR_PD,01.05.2026
373756,31.08.2024,120140004013,NaN,46538725,OBZ_D73433843865,10475,Кредиты в нацвалюте юридическим лицам с возмож...,26.08.2024,26.08.2026,NaN,24 месяца,15,355,KZT,4687000,"25,4","28,6",Актуален,NaN,4687000,NaN,"-13227,76",NaN,10.09.2024,"35094,99",16.02.2026,NaN,NaN,CR_INTER,01.02.2026
373757,31.08.2024,120140004013,NaN,46538725,OBZ_D73433843865,10475,Кредиты в нацвалюте юридическим лицам с возмож...,26.08.2024,26.08.2026,NaN,24 месяца,15,355,KZT,4687000,"25,4","28,6",Актуален,NaN,4687000,NaN,"-13227,76",NaN,10.09.2024,"157335,09",15.11.2024,NaN,NaN,CR_PD,01.11.2024
373758,31.08.2024,120140004013,NaN,46538725,OBZ_D73433843865,10475,Кредиты в нацвалюте юридическим лицам с возмож...,26.08.2024,26.08.2026,NaN,24 месяца,15,355,KZT,4687000,"25,4","28,6",Актуален,NaN,4687000,NaN,"-13227,76",NaN,10.09.2024,"15148,26",15.06.2026,NaN,NaN,CR_INTER,01.06.2026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18369867,31.08.2024,141040026073,NaN,46520045,OBZ_D73325588743,10475,Кредиты в нацвалюте юридическим лицам с возмож...,23.08.2024,23.08.2026,NaN,24 месяца,23,366,KZT,12471000,"27,1","30,7",Актуален,NaN,12471000,NaN,"-65715,24",NaN,14.09.2024,"485811,45",23.06.2025,NaN,NaN,CR_PD,01.06.2025
18369868,31.08.2024,141040026073,NaN,46520045,OBZ_D73325588743,10475,Кредиты в нацвалюте юридическим лицам с возмож...,23.08.2024,23.08.2026,NaN,24 месяца,23,366,KZT,12471000,"27,1","30,7",Актуален,NaN,12471000,NaN,"-65715,24",NaN,14.09.2024,"531182,83",23.10.2025,NaN,NaN,CR_PD,01.10.2025
18369869,31.08.2024,141040026073,NaN,46520045,OBZ_D73325588743,10475,Кредиты в нацвалюте юридическим лицам с возмож...,23.08.2024,23.08.2026,NaN,24 месяца,23,366,KZT,12471000,"27,1","30,7",Актуален,NaN,12471000,NaN,"-65715,24",NaN,14.09.2024,"559466,2",23.12.2025,NaN,NaN,CR_PD,01.12.2025
18369870,31.08.2024,141040026073,NaN,46520045,OBZ_D73325588743,10475,Кредиты в нацвалюте юридическим лицам с возмож...,23.08.2024,23.08.2026,NaN,24 месяца,23,366,KZT,12471000,"27,1","30,7",Актуален,NaN,12471000,NaN,"-65715,24",NaN,14.09.2024,"149161,39",23.09.2025,NaN,NaN,CR_INTER,01.09.2025


In [8]:
import numpy as np
def map_type_business(row):
    if pd.isnull(row['SUBJECT_BUSINESS_TYPE_NAME']):
        return np.nan
    elif 'Корпоративный бизнес' in row['SUBJECT_BUSINESS_TYPE_NAME']:
        return 'КБ'
    elif row['SUBJECT_BUSINESS_TYPE_NAME'] in ['Малый бизнес', 'Средний бизнес']:
        return 'МСБ'
    else:
        return row['SUBJECT_BUSINESS_TYPE_NAME']

# Apply the function to create the new column
avtotransport['type_business'] = avtotransport.apply(map_type_business, axis=1)

In [9]:
avtotransport['type_business'].value_counts()

type_business
МСБ                     15267365
КБ                       3544670
Дочерние организации       40930
Розничный бизнес             130
Name: count, dtype: int64

In [28]:
avtotransport

,DATE_VALUE,SUBJECT_IIN_BIN,SUBJECT_BUSINESS_TYPE_NAME,CONTRACT_ID,CONTRACT_NUMBER,CONTRACT_PRODUCT_CODE,CONTRACT_PRODUCT_NAME,CONTRACT_OPEN_DATE,CONTRACT_END_DATE,CONTRACT_LIQUIDATE_DATE,REMAIN_TERM,REPAYMENT_DAY,CONTRACT_DURATION,CURRENCY_CODE,CONTRACT_SUM_NAT,INTEREST_RATE,EFF_INTEREST_RATE,CONTRACT_STATUS_NAME,EXP_PD_OVRD_CNT,CR_BAL_NAT,CR_EXP_PD_NAT,CR_INT_CH_NAT,CR_EXP_IN_NAT,DATE_VALUE.1,SHEDULED_SUM_NAT,PLAN_DATE,FACT_DATE,STATUS,PAY_TYPE,PLAN_DATE_M,type_business
0,31.08.2024,050940002443,Корпоративный бизнес 2,43613429,KD 01-15-03-152,10281,Кредиты в инвалюте c возможностью досрочного п...,11.07.2023,28.09.2029,NaN,2271 Дней,11,1154,EUR,"1000728307,8","8,5",9,Актуален,NaN,"1101873200,22",NaN,"-108488597,7",NaN,10.09.2024,"5346153,92",11.06.2026,NaN,NaN,CR_INTER,01.06.2026,КБ
1,31.08.2024,050940002443,Корпоративный бизнес 2,43613429,KD 01-15-03-152,10281,Кредиты в инвалюте c возможностью досрочного п...,11.07.2023,28.09.2029,NaN,2271 Дней,11,1154,EUR,"1000728307,8","8,5",9,Актуален,NaN,"1101873200,22",NaN,"-108488597,7",NaN,10.09.2024,"17814746,59",11.12.2025,NaN,NaN,CR_PD,01.12.2025,КБ
2,31.08.2024,050940002443,Корпоративный бизнес 2,43613429,KD 01-15-03-152,10281,Кредиты в инвалюте c возможностью досрочного п...,11.07.2023,28.09.2029,NaN,2271 Дней,11,1154,EUR,"1000728307,8","8,5",9,Актуален,NaN,"1101873200,22",NaN,"-108488597,7",NaN,10.09.2024,"17814746,59",13.03.2028,NaN,NaN,CR_PD,01.03.2028,КБ
3,31.08.2024,050940002443,Корпоративный бизнес 2,43613429,KD 01-15-03-152,10281,Кредиты в инвалюте c возможностью досрочного п...,11.07.2023,28.09.2029,NaN,2271 Дней,11,1154,EUR,"1000728307,8","8,5",9,Актуален,NaN,"1101873200,22",NaN,"-108488597,7",NaN,10.09.2024,"6595417,82",13.10.2025,NaN,NaN,CR_INTER,01.10.2025,КБ
4,31.08.2024,050940002443,Корпоративный бизнес 2,43613429,KD 01-15-03-152,10281,Кредиты в инвалюте c возможностью досрочного п...,11.07.2023,28.09.2029,NaN,2271 Дней,11,1154,EUR,"1000728307,8","8,5",9,Актуален,NaN,"1101873200,22",NaN,"-108488597,7",NaN,10.09.2024,"1665678,17",13.11.2028,NaN,NaN,CR_INTER,01.11.2028,КБ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18853810,31.08.2024,831008300940,Малый бизнес,44143436,D1864458481.823007,10476,Кредиты в нацвалюте физическим лицам с возможн...,22.09.2023,22.09.2025,NaN,24 месяца,15,355,KZT,12000000,"29,38","33,6",Актуален,NaN,"7308339,43",NaN,"-2699147,1",NaN,14.09.2024,"560319,31",17.03.2025,NaN,NaN,CR_PD,01.03.2025,МСБ
18853811,31.08.2024,831008300940,Малый бизнес,44143436,D1864458481.823007,10476,Кредиты в нацвалюте физическим лицам с возможн...,22.09.2023,22.09.2025,NaN,24 месяца,15,355,KZT,12000000,"29,38","33,6",Актуален,NaN,"7308339,43",NaN,"-2699147,1",NaN,14.09.2024,"539338,27",17.02.2025,NaN,NaN,CR_PD,01.02.2025,МСБ
18853812,31.08.2024,831008300940,Малый бизнес,44143436,D1864458481.823007,10476,Кредиты в нацвалюте физическим лицам с возможн...,22.09.2023,22.09.2025,NaN,24 месяца,15,355,KZT,12000000,"29,38","33,6",Актуален,NaN,"7308339,43",NaN,"-2699147,1",NaN,14.09.2024,"151953,64",17.12.2024,NaN,NaN,CR_INTER,01.12.2024,МСБ
18853813,31.08.2024,831008300940,Малый бизнес,44143436,D1864458481.823007,10476,Кредиты в нацвалюте физическим лицам с возможн...,22.09.2023,22.09.2025,NaN,24 месяца,15,355,KZT,12000000,"29,38","33,6",Актуален,NaN,"7308339,43",NaN,"-2699147,1",NaN,14.09.2024,"542494,17",15.01.2025,NaN,NaN,CR_PD,01.01.2025,МСБ


In [10]:
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['PLAN_DATE'] = avtotransport['PLAN_DATE'].dt.strftime('%d.%m.%Y')
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['REPORT_DATE']='01.09.2024'
avtotransport['REPORT_DATE'] = pd.to_datetime(avtotransport['REPORT_DATE'], format='%d.%m.%Y')

# Calculate the time difference in years
avtotransport['time_to_mat'] = (avtotransport['PLAN_DATE'] - avtotransport['REPORT_DATE']).dt.days / 365
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].str.replace(',','.')
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].astype(float)
avtotransport['disc_fact']=1/(1+avtotransport.EFF_INTEREST_RATE/100)**avtotransport.time_to_mat
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].str.replace(',','.')
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].astype(float)
avtotransport['MP*disc_fact']=avtotransport.SHEDULED_SUM_NAT*avtotransport.disc_fact
avtotransport['MP*disc_fact*time']=avtotransport['MP*disc_fact']*avtotransport.time_to_mat
avtotransport['CR_BAL_NAT']=avtotransport['CR_BAL_NAT'].str.replace(',','.')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].replace('', '0')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].astype(float)

avtotransport_number=avtotransport.groupby(['type_business','CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
avtotransport_number['duration']=avtotransport_number['MP*disc_fact*time']/avtotransport_number['MP*disc_fact']
avtotransport_number['mod_dur']=avtotransport_number['duration']/(1+avtotransport.EFF_INTEREST_RATE/100)
avtotransport_number.CR_BAL_NAT=avtotransport_number.CR_BAL_NAT*(-1)
avtotransport_number['weighted']=avtotransport_number.mod_dur*avtotransport_number.CR_BAL_NAT
avtotransport_number['weighted']=avtotransport_number['weighted'].fillna(0)

In [11]:
avtotransport_number.type_business.value_counts()

type_business
МСБ                     75280
КБ                      16632
Дочерние организации      239
Розничный бизнес            2
Name: count, dtype: int64

In [12]:
doch=avtotransport_number[avtotransport_number['type_business']=='Дочерние организации']
mcb=avtotransport_number[avtotransport_number['type_business']=='МСБ']
roznica=avtotransport_number[avtotransport_number['type_business']=='Розничный бизнес']
kb=avtotransport_number[avtotransport_number['type_business']=='КБ']


In [13]:
print(sum(doch['weighted'])/sum(doch.CR_BAL_NAT))
print(sum(mcb['weighted'])/sum(mcb.CR_BAL_NAT))
print(sum(roznica['weighted'])/sum(roznica.CR_BAL_NAT))
print(sum(kb['weighted'])/sum(kb.CR_BAL_NAT))


1.1703895953280008
0.9258270289665579
0.3302840615911633
1.2334238124819836


In [38]:
os.chdir(fr"C:\Users\00059700\Desktop\Займы\new_files\Новая папка")

## POTREB 1

In [42]:
avtotransport=pd.read_csv('potreb_1.csv',sep=';',dtype=str)


In [43]:
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['PLAN_DATE'] = avtotransport['PLAN_DATE'].dt.strftime('%d.%m.%Y')
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['REPORT_DATE']='01.09.2024'
avtotransport['REPORT_DATE'] = pd.to_datetime(avtotransport['REPORT_DATE'], format='%d.%m.%Y')

# Calculate the time difference in years
avtotransport['time_to_mat'] = (avtotransport['PLAN_DATE'] - avtotransport['REPORT_DATE']).dt.days / 365
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].str.replace(',','.')
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].astype(float)
avtotransport['disc_fact']=1/(1+avtotransport.EFF_INTEREST_RATE/100)**avtotransport.time_to_mat
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].str.replace(',','.')
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].astype(float)
avtotransport['MP*disc_fact']=avtotransport.SHEDULED_SUM_NAT*avtotransport.disc_fact
avtotransport['MP*disc_fact*time']=avtotransport['MP*disc_fact']*avtotransport.time_to_mat
avtotransport['CR_BAL_NAT']=avtotransport['CR_BAL_NAT'].str.replace(',','.')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].replace('', '0')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].astype(float)

avtotransport_number=avtotransport.groupby(['RS_CR_CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
avtotransport_number['duration']=avtotransport_number['MP*disc_fact*time']/avtotransport_number['MP*disc_fact']
avtotransport_number['mod_dur']=avtotransport_number['duration']/(1+avtotransport.EFF_INTEREST_RATE/100)
avtotransport_number.CR_BAL_NAT=avtotransport_number.CR_BAL_NAT*(-1)
avtotransport_number['weighted']=avtotransport_number.mod_dur*avtotransport_number.CR_BAL_NAT
avtotransport_number['weighted']=avtotransport_number['weighted'].fillna(0)
sum(avtotransport_number['weighted'])/sum(avtotransport_number.CR_BAL_NAT)

2.1336353939325443

## POTREB 2

In [45]:
avtotransport=pd.read_csv('potreb_2.csv',sep=';',dtype=str)

avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['PLAN_DATE'] = avtotransport['PLAN_DATE'].dt.strftime('%d.%m.%Y')
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['REPORT_DATE']='01.09.2024'
avtotransport['REPORT_DATE'] = pd.to_datetime(avtotransport['REPORT_DATE'], format='%d.%m.%Y')

# Calculate the time difference in years
avtotransport['time_to_mat'] = (avtotransport['PLAN_DATE'] - avtotransport['REPORT_DATE']).dt.days / 365
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].str.replace(',','.')
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].astype(float)
avtotransport['disc_fact']=1/(1+avtotransport.EFF_INTEREST_RATE/100)**avtotransport.time_to_mat
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].str.replace(',','.')
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].astype(float)
avtotransport['MP*disc_fact']=avtotransport.SHEDULED_SUM_NAT*avtotransport.disc_fact
avtotransport['MP*disc_fact*time']=avtotransport['MP*disc_fact']*avtotransport.time_to_mat
avtotransport['CR_BAL_NAT']=avtotransport['CR_BAL_NAT'].str.replace(',','.')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].replace('', '0')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].astype(float)

avtotransport_number=avtotransport.groupby(['RS_CR_CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
avtotransport_number['duration']=avtotransport_number['MP*disc_fact*time']/avtotransport_number['MP*disc_fact']
avtotransport_number['mod_dur']=avtotransport_number['duration']/(1+avtotransport.EFF_INTEREST_RATE/100)
avtotransport_number.CR_BAL_NAT=avtotransport_number.CR_BAL_NAT*(-1)
avtotransport_number['weighted']=avtotransport_number.mod_dur*avtotransport_number.CR_BAL_NAT
avtotransport_number['weighted']=avtotransport_number['weighted'].fillna(0)
sum(avtotransport_number['weighted'])/sum(avtotransport_number.CR_BAL_NAT)

0.8587755631048739

## POTREB 4

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
import os 
os.chdir(fr"C:\Users\00059700\Desktop\Займы\new_files\Новая папка")



In [3]:

avtotransport=pd.read_csv('potreb_4.csv',sep=';',dtype=str)

avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['PLAN_DATE'] = avtotransport['PLAN_DATE'].dt.strftime('%d.%m.%Y')
avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
avtotransport['REPORT_DATE']='01.09.2024'
avtotransport['REPORT_DATE'] = pd.to_datetime(avtotransport['REPORT_DATE'], format='%d.%m.%Y')

# Calculate the time difference in years
avtotransport['time_to_mat'] = (avtotransport['PLAN_DATE'] - avtotransport['REPORT_DATE']).dt.days / 365
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].str.replace(',','.')
avtotransport['EFF_INTEREST_RATE']=avtotransport['EFF_INTEREST_RATE'].astype(float)
avtotransport['disc_fact']=1/(1+avtotransport.EFF_INTEREST_RATE/100)**avtotransport.time_to_mat
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].str.replace(',','.')
avtotransport['SHEDULED_SUM_NAT']=avtotransport['SHEDULED_SUM_NAT'].astype(float)
avtotransport['MP*disc_fact']=avtotransport.SHEDULED_SUM_NAT*avtotransport.disc_fact
avtotransport['MP*disc_fact*time']=avtotransport['MP*disc_fact']*avtotransport.time_to_mat
avtotransport['CR_BAL_NAT']=avtotransport['CR_BAL_NAT'].str.replace(',','.')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].replace('', '0')
avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].astype(float)

avtotransport_number=avtotransport.groupby(['RS_CR_CONTRACT_ID','EFF_INTEREST_RATE','CR_BAL_NAT'],as_index=False).agg({'MP*disc_fact':'sum', 'MP*disc_fact*time':'sum','CR_BAL_NAT':'last'})
avtotransport_number['duration']=avtotransport_number['MP*disc_fact*time']/avtotransport_number['MP*disc_fact']
avtotransport_number['mod_dur']=avtotransport_number['duration']/(1+avtotransport.EFF_INTEREST_RATE/100)
avtotransport_number.CR_BAL_NAT=avtotransport_number.CR_BAL_NAT*(-1)
avtotransport_number['weighted']=avtotransport_number.mod_dur*avtotransport_number.CR_BAL_NAT
avtotransport_number['weighted']=avtotransport_number['weighted'].fillna(0)
sum(avtotransport_number['weighted'])/sum(avtotransport_number.CR_BAL_NAT)

1.1229820944202427

In [4]:
import csv
 
 
def write_csv(filename, data, header=None):
    with open(filename, 'w') as file:
        writer = csv.writer(file)
        if header:
            writer.writerow(header)
        writer.writerows(data)
 
 
def split_csv(filename, num_rows, has_header=True):
    name, extension = filename.split('.')
    file_no = 1
    chunk = []
    row_count = 0
    header = ''
 
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if has_header:
                header = row
                has_header = False
                continue
            chunk.append(row)
            row_count += 1
            if row_count >= num_rows:
                write_csv(f'{name}-{file_no}.{extension}', chunk, header)
                chunk = []
                file_no += 1
                row_count = 0
        if chunk:
            write_csv(f'{name}-{file_no}.{extension}', chunk, header)



split_csv('potreb_3.csv', 10000000)



In [6]:
potrebbb=pd.read_csv('potreb_3-1.csv',dtype=str,sep=';')

In [11]:
import csv

data=pd.read_csv("potreb_3-1(new).csv",dtype=str,sep=';')

## POTREB 3 = 1.1655959364723067

In [13]:
import pandas as pd
import csv

# Function to remove double quotes from all values in a CSV file
def remove_double_quotes(input_csv, output_csv):
    with open(input_csv, 'r', newline='') as infile, open(output_csv, 'w', newline='') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)
        
        for row in reader:
            new_row = [value.replace('"', '') for value in row]
            writer.writerow(new_row)

# List of CSV files to process
file_list = ['potreb_3-1.csv', 'potreb_3-2.csv', 'potreb_3-3.csv', 'potreb_3-4.csv', 'potreb_3-5.csv', 'potreb_3-6.csv', 'potreb_3-7.csv', 'potreb_3-8.csv']

for file in file_list:
    input_file = file
    output_file = file.replace('.csv', '_no_quotes.csv')
    
    # Remove double quotes from the input file and save the result to the output file
    remove_double_quotes(input_file, output_file)
    
    # Read the CSV file and apply duration calculation
    avtotransport = pd.read_csv(output_file, sep=';', dtype=str)
    
    # Perform the duration calculation steps
    avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
    avtotransport['PLAN_DATE'] = avtotransport['PLAN_DATE'].dt.strftime('%d.%m.%Y')
    avtotransport['PLAN_DATE'] = pd.to_datetime(avtotransport['PLAN_DATE'], format='%d.%m.%Y')
    avtotransport['REPORT_DATE'] = '01.09.2024'
    avtotransport['REPORT_DATE'] = pd.to_datetime(avtotransport['REPORT_DATE'], format='%d.%m.%Y')

    avtotransport['time_to_mat'] = (avtotransport['PLAN_DATE'] - avtotransport['REPORT_DATE']).dt.days / 365
    avtotransport['EFF_INTEREST_RATE'] = avtotransport['EFF_INTEREST_RATE'].str.replace(',', '.')
    avtotransport['EFF_INTEREST_RATE'] = avtotransport['EFF_INTEREST_RATE'].astype(float)
    avtotransport['disc_fact'] = 1 / (1 + avtotransport['EFF_INTEREST_RATE'] / 100) ** avtotransport['time_to_mat']
    avtotransport['SHEDULED_SUM_NAT'] = avtotransport['SHEDULED_SUM_NAT'].str.replace(',', '.')
    avtotransport['SHEDULED_SUM_NAT'] = avtotransport['SHEDULED_SUM_NAT'].astype(float)
    avtotransport['MP*disc_fact'] = avtotransport['SHEDULED_SUM_NAT'] * avtotransport['disc_fact']
    avtotransport['MP*disc_fact*time'] = avtotransport['MP*disc_fact'] * avtotransport['time_to_mat']
    avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].str.replace(',', '')
    avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].replace('', '0')
    avtotransport['CR_BAL_NAT'] = avtotransport['CR_BAL_NAT'].astype(float)

    avtotransport_number = avtotransport.groupby(['RS_CR_CONTRACT_ID', 'EFF_INTEREST_RATE', 'CR_BAL_NAT'], as_index=False).agg({'MP*disc_fact': 'sum', 'MP*disc_fact*time': 'sum', 'CR_BAL_NAT': 'last'})
    avtotransport_number['duration'] = avtotransport_number['MP*disc_fact*time'] / avtotransport_number['MP*disc_fact']
    avtotransport_number['mod_dur'] = avtotransport_number['duration'] / (1 + avtotransport['EFF_INTEREST_RATE'] / 100)
    avtotransport_number.CR_BAL_NAT = avtotransport_number.CR_BAL_NAT * (-1)
    avtotransport_number['weighted'] = avtotransport_number['mod_dur'] * avtotransport_number['CR_BAL_NAT']
    avtotransport_number['weighted'] = avtotransport_number['weighted'].fillna(0)
    duration = sum(avtotransport_number['weighted']) / sum(avtotransport_number['CR_BAL_NAT'])
    
    print(f'Duration calculated for {file}: {duration}')

Duration calculated for potreb_3-1.csv: 1.1457510793125218
Duration calculated for potreb_3-2.csv: 1.1543320227350125
Duration calculated for potreb_3-3.csv: 1.160055682777195
Duration calculated for potreb_3-4.csv: 1.1581228092219864
Duration calculated for potreb_3-5.csv: 1.1552795341791322
Duration calculated for potreb_3-6.csv: 1.169039058439314
Duration calculated for potreb_3-7.csv: 1.1740250446061036
Duration calculated for potreb_3-8.csv: 1.2081622605071867


## POTREB (1+2+3+4) / 4 = 1.32

In [16]:
(1.1655959364723067+ 1.1229820944202427 + 0.8587755631048739 + 2.1336353939325443) /4


1.320247246982492

## RS

### АВТОТРАНСПОРТ = 1.9

### ИПОТЕКА = 4.2 

### ПОДЗАЛОГ = 1.9

### ОБРАЗОВАНИЕ = 0.48

### ПЕНСИЯ = 0.58

### ПОТРЕБ = 1.32

## BS

### ДОЧЕРНИЕ КОМПАНИИ = 1.17

### МСБ = 0.92

### РОЗНИЦА = 0.33

### КБ= 1.23